In [62]:
import os
import time
import bs4
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

#checking for the writing rights
try:
    with open("write_test.txt", "w") as f:
        f.write("test")
    os.remove("write_test.txt")
    print("Write permission: OK")
except Exception as e:
    print("Write permission error:", e)
    exit(1)

folderName = 'imageees'
if not os.path.isdir(folderName):
    os.mkdir(folderName)


def download_image(url, folderName, num):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            file_path = os.path.join(folderName, str(num) + ".jpg")
            with open(file_path, 'wb') as file:
                file.write(response.content)
            print(f"Downloaded: {file_path}")
        else:
            print(f"Failed to download image {num}: HTTP {response.status_code}")
    except Exception as e:
        print(f"Error downloading image {num}: {e}")

chromeDriverPath = r'C:\Users\swarn\OneDrive\Documents\Drivers\chromedriver.exe'
service = Service(chromeDriverPath)
driver = webdriver.Chrome(service=service)

search_URL = (
    "https://www.google.com/search?sca_esv=d821bfeee38a6cd8&hl=en&sxsrf=AE3TifNYwSW79FQ8Tu7aJaYUY_c2TYq_FQ:1749469504270"
    "&q=bus&udm=2&fbs=AIIjpHxU7SXXniUZfeShr2fp4giZ1Y6MJ25_tmWITc7uy4KIeoJTKjrFjVxydQWqI2NcOhYPURIv2wPgv_w_sE_0Sc6QogS5"
    "TvEDp7UpbJYBVowPixTj9Ih7YqadN2jzeCfRt-pBAaHrlSdDKXkA6CS-ow1t3vJ2olVeJzygke7ovRG9qdf9qZD_BwqoEvrwG7MxmUaxp4MgDYkMmxrTJ09nI1QH5nBzXg"
    "&sa=X&ved=2ahUKEwjAxuPioeSNAxVvsFYBHV7TDU8QtKgLegQIEhAB&biw=1536&bih=730&dpr=1.25"
)
driver.get(search_URL)
input("Waiting for user input to start... (scroll and load images if needed)")

driver.execute_script("window.scrollTo(0, 0);")


page_html = driver.page_source
pageSoup = bs4.BeautifulSoup(page_html, 'html.parser')

containers = pageSoup.findAll('div', {
    'class': "eA0Zlc WghbWd FnEtTd mkpRId m3LIae RLdvSe qyKxnc ivg-i PZPZlf GMCzAd"
})
len_containers = len(containers)
print(f"Found {len_containers} image containers")


✅ Write permission: OK


📌 Waiting for user input to start... (scroll and load images if needed) 


🔍 Found 100 image containers


In [70]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time

wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "GMCzAd")))

# Find all containers
containers = driver.find_elements(By.CLASS_NAME, "GMCzAd")
len_containers = len(containers)

for i in range(1, 20):
    if i % 25 == 0:
        continue

    xPath = f'(//div[contains(@class, "GMCzAd")])[{i}]'

    try:
        container = driver.find_element(By.XPATH, xPath)
        lowResImageEle = container.find_element(By.XPATH, './/img[1]')
        lowResImageURL = lowResImageEle.get_attribute("src")

        container.click()

        startTime = time.time()
        highResURL = lowResImageURL  # default to low-res

        while True:
            try:
                imageEle = driver.find_element(By.XPATH, '//*[@id="Sva75c"]/div[2]/div[2]/div/div[2]/c-wiz/div/div[2]/div[2]/a/img[1]')
                newURL = imageEle.get_attribute("src")
                if newURL != lowResImageURL:
                    highResURL = newURL
                    print(f"High-res image loaded for container {i}: {highResURL}")
                    break
                elif time.time() - startTime > 10:
                    print(f"Timeout, using low-res image for container {i}")
                    break
                time.sleep(0.2)
            except NoSuchElementException:
                if time.time() - startTime > 10:
                    print(f"High-res image element not found for container {i}, using low-res")
                    break
                time.sleep(0.2)
        download_image(highResURL, folderName, i)

    except NoSuchElementException:
        print(f"Element not found for container {i}, skipping...")
    except Exception as e:
        print(f"Unexpected error at container {i}: {e}")


✅ High-res image loaded for container 1: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSQQNaNJlLH0d7ELZtZCtPOt8kzQJA2QqvncQ&s
✅ Downloaded: imageees\1.jpg
✅ High-res image loaded for container 2: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSQQNaNJlLH0d7ELZtZCtPOt8kzQJA2QqvncQ&s
✅ Downloaded: imageees\2.jpg
✅ High-res image loaded for container 3: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQjOCUZTVjdxeWMq5QmBWIduW0HoA3UP_ALsA&s
✅ Downloaded: imageees\3.jpg
✅ High-res image loaded for container 4: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR18RCMbD6ka4tt8CYGm3Ef4usXfnjbT6ttGw&s
✅ Downloaded: imageees\4.jpg
✅ High-res image loaded for container 5: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQrpbaD5NYIODfWinQIrbVnav4IJCiw2dPydQ&s
✅ Downloaded: imageees\5.jpg
✅ High-res image loaded for container 6: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTuV8GBSycrg2YxlKrhoFE4j0pTd6v4u5-7ew&s
✅ Downloaded: imageees\6.jpg
✅ High-res image loade